In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")

from influence.dataset import DataSet
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import base


fpath_titanic = "/home/eolus/Desktop/Dauphine/datamining/projets/blackBox/data/train.csv"
train_df = pd.read_csv(fpath_titanic)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
def extract_prefix(name):
    import re
    try:
        return re.search('(Mr\.)|(Mrs\.)|(Miss\.)', name).group()
    except:
        return ""


train_df['Prefix'] = train_df.Name.apply(extract_prefix)

for cat_col in ['Sex', 'Embarked', 'Prefix' ]:
    train_df[cat_col] = pd.factorize(train_df[cat_col])[0]
    
train_df['Age'].fillna(train_df.Age.mean(), inplace=True)  

In [3]:
features = ['Prefix', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

X = np.array(train_df[features])
y = np.array((train_df.Survived > 0).astype('int32'))

In [4]:
# Scale
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Need to train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
def get_model(X_train, X_test, y_train, y_test):
    
    # Create dataset objects
    import sys
    sys.path.append("..")

    #from influence indataset as dataset
    from influence.dataset import DataSet
    import numpy as np
    lr_train = DataSet(X_train, np.array(y_train, dtype=int))
    lr_test = DataSet(X_test, np.array(y_test, dtype=int))
    lr_validation = None

    import tensorflow as tf
    from tensorflow.contrib.learn.python.learn.datasets import base
    lr_data_sets = base.Datasets(train=lr_train, validation=lr_validation, test=lr_test)

    from influence.binaryLogisticRegressionWithLBFGS import BinaryLogisticRegressionWithLBFGS

    num_classes = 2
    #input_dim = len(features)
    input_dim = len(X_test.T)

    weight_decay = 0.01
    batch_size = 100
    initial_learning_rate = 0.001 
    keep_probs = None
    decay_epochs = [1000, 10000]
    max_lbfgs_iter = 1000

    tf.reset_default_graph()

    tf_model = BinaryLogisticRegressionWithLBFGS(
        input_dim=input_dim,
        weight_decay=weight_decay,
        max_lbfgs_iter=max_lbfgs_iter,
        num_classes=num_classes, 
        batch_size=batch_size,
        data_sets=lr_data_sets,
        initial_learning_rate=initial_learning_rate,
        keep_probs=keep_probs,
        decay_epochs=decay_epochs,
        mini_batch=False,
        train_dir='tmp',
        log_dir='tmp',
        model_name='titanic')

    tf_model.train()
    return tf_model

In [6]:
def plot(x, y):
    import matplotlib.pyplot as plt
    plt.plot( x, y, '-')
    plt.show()


# Get top influential train points (I_up_loss)
def get_top_train_influence(tf_model):
    """
    Approximate most influential train points for a test point
    idx : index of test point
    """
    num_train = len(tf_model.data_sets.train.labels)
    influences = tf_model.get_influence_on_test_loss(
        np.arange(len(tf_model.data_sets.test.labels)), 
        np.arange(len(tf_model.data_sets.train.labels)),
        force_refresh=True) * num_train
    influences_sorted = sorted(enumerate(influences),
                               key=lambda x:x[1],
                               reverse=True)
    return influences_sorted

In [10]:
def retrain_most_influential_train(top_train_influence):

    scores = []
    for i in range(3, 101, 1):

        print("WITH {}% MOST INFLUENTIALTRAINING POINTS:".format(i))
        
        top_n = [f[0] for f in top_train_influence[:int(i*len(top_train_influence)/100)]]
        print("TOP N:", len(top_n))
        
        from influence.dataset import DataSet
        lr_train = DataSet(X_train[top_n,:], np.array(y_train[top_n], dtype=int))
        lr_test = DataSet(X_test, np.array(y_test, dtype=int))
        lr_validation = None
        lr_data_sets = base.Datasets(train=lr_train, validation=lr_validation, test=lr_test)


        from influence.binaryLogisticRegressionWithLBFGS import BinaryLogisticRegressionWithLBFGS

        num_classes = 2
        input_dim = len(X_train.T)

        weight_decay = 0.01
        batch_size = 100
        initial_learning_rate = 0.001 
        keep_probs = None
        decay_epochs = [1000, 10000]
        max_lbfgs_iter = 1000

        tf.reset_default_graph()

        tf_model = BinaryLogisticRegressionWithLBFGS(
            input_dim=input_dim,
            weight_decay=weight_decay,
            max_lbfgs_iter=max_lbfgs_iter,
            num_classes=num_classes, 
            batch_size=batch_size,
            data_sets=lr_data_sets,
            initial_learning_rate=initial_learning_rate,
            keep_probs=keep_probs,
            decay_epochs=decay_epochs,
            mini_batch=False,
            train_dir='tmp',
            log_dir='tmp',
            model_name='titanic')

        tf_model.train()

        preds_p = tf_model.get_preds().tolist()
        preds = [1 if el[0] < 0.5 else 0 for el in preds_p]
        ref = tf_model.data_sets.test.labels

        same = 0
        for test_point in zip(preds, ref):
            if test_point[0] == test_point[1]:
                same += 1
        score = same / len(ref)
        print("SCORE = ", score)
        print()

        scores.append((i, score))
        
    x = [s[0] for s in scores]
    y = [s[1] for s in scores]
    return x, y

In [11]:
#top_train_influence = get_top_train_influence(tf_model)
#x, y = retrain_most_influential_train(top_train_influence)
#plot(x, y)

In [12]:
X_train = X_train[:, 0:3]
X_test = X_test[:, 0:3]
tf_model = get_model(X_train, X_test, y_train, y_test)
top_train_influence = get_top_train_influence(tf_model)
retrain_most_influential_train(top_train_influence)

Total number of parameters: 3
Using normal model
LBFGS training took [7] iter.
After training with LBFGS: 
Optimization terminated successfully.
         Current function value: -0.007806
         Iterations: 4
         Function evaluations: 5
         Gradient evaluations: 8
         Hessian evaluations: 7
WITH 3% MOST INFLUENTIALTRAINING POINTS:
TOP N: 17
Total number of parameters: 3
Using normal model
LBFGS training took [11] iter.
After training with LBFGS: 
SCORE =  0.8033898305084746

WITH 4% MOST INFLUENTIALTRAINING POINTS:
TOP N: 23
Total number of parameters: 3
Using normal model
LBFGS training took [11] iter.
After training with LBFGS: 
SCORE =  0.8033898305084746

WITH 5% MOST INFLUENTIALTRAINING POINTS:
TOP N: 29
Total number of parameters: 3
Using normal model
LBFGS training took [11] iter.
After training with LBFGS: 
SCORE =  0.8033898305084746

WITH 6% MOST INFLUENTIALTRAINING POINTS:
TOP N: 35
Total number of parameters: 3
Using normal model
LBFGS training took [11] it

([3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100],
 [0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,
  0.8033898305084746,